# Velib

## Config

In [5]:
API_KEY = "a44bff4661b1435a625ea7b661a3f6410386bcf3"
BOOTSTRAP_SERVERS = 'kafka:9092'
# For several servers
#BOOTSTRAP_SERVERS = ['localhost:9092', 'localhost:9093']

## Install package `kafka-python`

In [3]:
import sys
!conda install --yes --prefix {sys.prefix} kafka-python

# Or via pip
#import sys
#!{sys.executable} -m pip install kafka-python

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



## Kafka Producer (velib-get-stations.py)

In [9]:
import json
import time
import urllib.request

from kafka import KafkaProducer

#API_KEY = "XXX" # FIXME Set your own API key here
API_KEY = "a44bff4661b1435a625ea7b661a3f6410386bcf3"
url = "https://api.jcdecaux.com/vls/v1/stations?apiKey={}".format(API_KEY)
producer = KafkaProducer(bootstrap_servers=BOOTSTRAP_SERVERS)
stations = {}

while True:
    response = urllib.request.urlopen(url)
    stations_upd8 = json.loads(response.read().decode())
    for station in stations_upd8:
        station_number = station["number"]
        contract = station["contract_name"]
        available_bikes = station["available_bikes"]

        if contract not in stations:
            stations[contract] = {}

        city_stations = stations[contract]

        if station_number not in city_stations:
            city_stations[station_number] = available_bikes
        else:
            count_diff = available_bikes - city_stations[station_number]
            empty_shift = (available_bikes == 0  or city_stations[station_number] == 0) and count_diff != 0

            if empty_shift:
                city_stations[station_number] = available_bikes
                producer.send("empty-stations", json.dumps(station).encode(),
                        key=contract.encode()) # dispatch each city always on same partition
                print("{} - {} : {} bikes available".format(station["number"], station["contract_name"], station["available_bikes"]))
    print("-------------")
#    print("{} Produced {} station records".format(time.time(), len(stations)))
    time.sleep(1)

-------------
178 - Valence : 0 bikes available
60 - Dublin : 1 bikes available
-------------
-------------
10091 - Lyon : 1 bikes available
6113 - Marseille : 1 bikes available
4023 - Lyon : 0 bikes available
4006 - Lyon : 1 bikes available
5107 - Marseille : 1 bikes available
6012 - Lyon : 1 bikes available
5005 - Lyon : 0 bikes available
10002 - Lyon : 1 bikes available
8004 - Lyon : 0 bikes available
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
-------------
112 - Dublin : 0 bikes available
-------------
-------------
10086 - Lyon : 1 bikes available
8004 - Lyon : 1 bikes available
----

KeyboardInterrupt: 

## Kafka Broker

In [ ]:
%%bash
./bin/zookeeper-server-start.sh ./config/zookeeper.properties
./bin/kafka-server-start.sh ./config/server.properties
./bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic velib-stations

## Kafka Consumer (velib-monitor-stations.py)

In [ ]:
import json
from kafka import KafkaConsumer

consumer = KafkaConsumer("empty-stations", bootstrap_servers=BOOTSTRAP_SERVERS, group_id="velib-monitor-stations")
cities = {}

for message in consumer:
    station = json.loads(message.value.decode())
    contract = station["contract_name"]
    available_bikes = station["available_bikes"]

    if contract not in cities:
        cities[contract] = 0

    if available_bikes == 0:
        cities[contract] += 1
        print("no more bikes at {} ({}). {} empty station(s) in this city".format(
            station["address"], contract, cities[contract]))
    elif cities[contract] != 0: # We don't know how many stations are empty at startup
        cities[contract] -= 1
        

## Modification de la durée de conservation des messages dans Kafka

Suppression d'un segment toutes les 4 secondes

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config retention.ms=4000

Modification de la longueur d'un segment

In [ ]:
! ./bin/kafka-configs.sh --zookeeper localhost:2181 --entity-type topics --entity-name velib-stations --alter --add-config segment.ms=2000